## Required imports

In [1]:
SAVE_GOOGLE_COLAB = False # Flag used to save results to a Google Drive when operating on Google Colab
import shutil
if SAVE_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [1]:
!pip install memtorch-1.1.0.tar.gz

Processing c:\users\jc299170\desktop\sst-reproducibility\memtorch-1.1.0.tar.gz
  Created wheel for memtorch: filename=memtorch-1.1.0-cp38-cp38-win_amd64.whl size=226549 sha256=cbd828611eb4bf1133c852874ceda53842c5ccf2243699214e513e8ad8c2526f
  Stored in directory: c:\users\jc299170\appdata\local\pip\cache\wheels\fc\26\61\6f750449683b8ec62006919e3ec6d44ac7d29df7275367763f
Successfully built memtorch
  Attempting uninstall: memtorch
    Found existing installation: memtorch 1.1.0
    Uninstalling memtorch-1.1.0:
      Successfully uninstalled memtorch-1.1.0


## Define and train the CNN

In [4]:
import torch
from torch.autograd import Variable
import memtorch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import torchvision
from mobilenetv2 import MobileNetV2


def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):        
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

set_all_seeds(0)
device = torch.device('cpu' if 'cpu' in memtorch.__version__ else 'cuda')
epochs = 100
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, num_workers=1)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=1)
model = MobileNetV2().to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)
best_accuracy = 0
# for epoch in range(0, epochs):
#     print('Epoch: [%d]\t\t' % (epoch + 1), end='')
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         optimizer.zero_grad()
#         output = model(data.to(device))
#         loss = criterion(output, target.to(device))
#         loss.backward()
#         optimizer.step()

#     scheduler.step()
#     model.eval()
#     accuracy = test(model, test_loader)
#     print('%2.2f%%' % accuracy)
#     if accuracy > best_accuracy:
#         print('Saving model...')
#         torch.save(model.state_dict(), 'trained_model.pt')
#         best_accuracy = accuracy

Files already downloaded and verified
Files already downloaded and verified


## Validate the CNN

In [5]:
import torch
import memtorch
from memtorch.utils import LoadCIFAR10
import numpy as np
from mobilenetv2 import MobileNetV2
import torchvision


def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

device = torch.device('cuda')
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=1)
model = MobileNetV2().to(device)
try:
    model.load_state_dict(torch.load('trained_model.pt'), strict=False)
    model.eval()
except:
    raise Exception('trained_model.pt has not been found.')

print('Test Set Accuracy: \t%2.2f%%' % test(model, test_loader))

Files already downloaded and verified
Test Set Accuracy: 	91.65%


## Convert the Model

In [6]:
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program

r_on = 2.00e4
r_off = 10.75e4
reference_memristor = memtorch.bh.memristor.VTEAM
reference_memristor_params = {'time_series_resolution': 1e-10,
                              'r_off': r_off,
                              'r_on': r_on}
model = MobileNetV2().to(device)
model.load_state_dict(torch.load('trained_model.pt'), strict=False)
patched_model = patch_model(copy.deepcopy(model),
                          memristor_model=reference_memristor,
                          memristor_model_params=reference_memristor_params,
                          module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                          mapping_routine=naive_map,
                          transistor=True,
                          programming_routine=None,
                          tile_shape=(128, 128),
                          max_input_voltage=0.3,
                          ADC_resolution=8,
                          ADC_overflow_rate=0.,
                          quant_method='linear')

Patched Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) -> bh.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Patched Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_channels=32, out_channels=32, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
Patched Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False) -> bh.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Patched Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_channels=32, out_channels=16, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
Patched Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_channels=32, out_channels=16, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
Patched Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_channels=16, out_channels=96, kernel_s

Patched Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=960, bias=False) -> bh.Conv2d(in_channels=960, out_channels=960, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Patched Conv2d(960, 160, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_channels=960, out_channels=160, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
Patched Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_channels=160, out_channels=960, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
Patched Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=960, bias=False) -> bh.Conv2d(in_channels=960, out_channels=960, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Patched Conv2d(960, 160, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_channels=960, out_channels=160, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
Patched Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False) -> bh.Conv2d(in_c

In [7]:
patched_model.tune_()

Tuned bh.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)). Coefficient of determination: 0.999886 [224128.421875, 0.000265]
Tuned bh.Conv2d(in_channels=320, out_channels=1280, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)). Coefficient of determination: 0.999920 [13060.810547, -0.000002]
Tuned bh.Linear(in_features=1280, out_features=10, bias=True). Coefficient of determination: 0.999961 [30545.607422, 0.000018]
Tuned bh.Conv2d(in_channels=32, out_channels=16, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)). Coefficient of determination: 0.999951 [107741.460938, -0.000006]
Tuned bh.Conv2d(in_channels=16, out_channels=24, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)). Coefficient of determination: 0.999949 [78823.843750, -0.000016]
Tuned bh.Conv2d(in_channels=144, out_channels=24, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)). Coefficient of determination: 0.999946 [49742.332031, -0.000029]
Tuned bh.Conv2d(in_channels=144, out_ch

In [ ]:
print('1')
print(test(patched_model, test_loader))

## Device endurance (gradual) simulations

In [ ]:
# import enum
# from enum import Enum, auto
# from memtorch.mn.Module import supported_module_parameters
# import math
# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from memtorch.mn.Module import patch_model
# from memtorch.map.Module import naive_tune
# from memtorch.map.Parameter import naive_map
# from memtorch.bh.crossbar.Program import naive_program
# from memtorch.bh.nonideality.NonIdeality import apply_nonidealities
# from memtorch.bh.crossbar.Crossbar import init_crossbar
# import copy
# from pprint import pprint
# from mobilenetv2 import MobileNetV2
# from scipy.interpolate import interp1d
# import torchvision


# def minimal_tune(model):
#     for i, (name, m) in enumerate(list(model.named_modules())):
#         if hasattr(m, 'tune'):
#             m.transform_output = lambda input: input
#             if isinstance(m, memtorch.mn.Conv2d):
#                 try:
#                     m.transform_output = naive_tune(m, (4, m.in_channels, 8, 8))
#                 except:
#                     pass
#             if isinstance(m, memtorch.mn.Linear):
#                 try:
#                     m.transform_output = naive_tune(m, (64, m.in_features))
#                 except:
#                     pass
                
#     return model.to(device)
    
# def update_patched_model(patched_model, model):
#     for i, (name, m) in enumerate(list(patched_model.named_modules())):
#         if isinstance(m, memtorch.mn.Conv2d) or isinstance(m, memtorch.mn.Linear):
#             pos_conductance_matrix, neg_conductance_matrix = naive_map(getattr(model, name).weight.data, r_on, r_off,scheme=memtorch.bh.Scheme.DoubleColumn)
#             m.crossbars[0].write_conductance_matrix(pos_conductance_matrix, transistor=True, programming_routine=None)
#             m.crossbars[1].write_conductance_matrix(neg_conductance_matrix, transistor=True, programming_routine=None)
#             m.weight.data = getattr(model, name).weight.data
            
#     return patched_model
    
# scale_input = interp1d([1.3, 1.9], [0, 1])
# def scale_p_0(p_0, p_1, v_stop, cell_size=10):
#     scaled_input = scale_input(v_stop)
#     x = 1.50
#     y = p_0 * math.exp(p_1 * cell_size)
#     k = math.log10(y) / (1 - (2 * scale_input(x) - 1) ** (2))
#     new_y = 10 ** (k * (1 - (2 * scaled_input - 1) ** (2)))
#     # Backsolve for p_0
#     p_0 = new_y / math.exp(p_1 * cell_size)
#     return p_0

# def gradual(input, cycle_count, p_1, p_2, p_3, cell_size):
#     p_0 = torch.log10(input)
#     threshold = p_1 * math.exp(p_2 * cell_size)
#     return torch.pow(10, (p_3 * cell_size * math.log10(cycle_count) + torch.log10(10 **  p_0) - p_3 * cell_size * math.log10(threshold)))
       
# def model_gradual(layer, cycle_count, v_stop):
#     cell_size = 10
#     convergence_point = 1e4
#     p_1_lrs = 1.0399076623425807
#     p_2_lrs = 0.9171208448973687
#     p_3_lrs = 0.0143551595777695
#     p_1_hrs = 4.3590883730463410
#     p_2_hrs = 0.7738077425228179
#     p_3_hrs = -0.018865423084966
#     p_1_lrs = scale_p_0(p_1_lrs, p_2_lrs, v_stop)
#     p_1_hrs = scale_p_0(p_1_hrs, p_2_hrs, v_stop)
#     threshold_lrs = p_1_lrs * math.exp(p_2_lrs * cell_size)
#     threshold_hrs = p_1_hrs * math.exp(p_2_hrs * cell_size)
#     for i in range(len(layer.crossbars)):
#         input = 1 / layer.crossbars[i].conductance_matrix
#         if input[input < convergence_point].nelement() > 0:
#             if cycle_count > threshold_lrs:
#                 input[input < convergence_point] = gradual(input[input < convergence_point], cycle_count, p_1_lrs, p_2_lrs, p_3_lrs, cell_size)
#         if input[input > convergence_point].nelement() > 0:
#             if cycle_count > threshold_hrs:
#                 input[input > convergence_point] = gradual(input[input > convergence_point], cycle_count, p_1_hrs, p_2_hrs, p_3_hrs, cell_size)
                
#         layer.crossbars[i].conductance_matrix = 1 / input

#     return layer

# def model_degradation(model, cycle_count, v_stop):
#     for i, (name, m) in enumerate(list(model.named_modules())):
#         if type(m) in supported_module_parameters.values():
#             setattr(model, name, model_gradual(m, cycle_count, v_stop)) # setattr(model.module, name, model_gradual(m, cycle_count, v_stop))

                    
#     return model

# device = torch.device('cuda')
# batch_size = 64
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
# train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, num_workers=1)
# test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=1)
# reference_memristor = memtorch.bh.memristor.VTEAM
# r_on = 4400
# r_off = 65000
# reference_memristor_params = {'time_series_resolution': 1e-10,
#                               'r_off': r_off,
#                               'r_on': r_on}
# times_to_reprogram = 10 ** np.arange(1, 10, dtype=np.float64)
# v_stop_values = np.linspace(1.3, 1.9, 10, endpoint=True)
# df = pd.DataFrame(columns=['times_reprogramed', 'v_stop', 'test_set_accuracy'])
# for time_to_reprogram in times_to_reprogram:
#     cycle_count = time_to_reprogram
#     for v_stop in v_stop_values:
#         print('time_to_reprogram: %f, v_stop: %f' % (time_to_reprogram, v_stop))
#         model = MobileNetV2().to(device)
#         model.load_state_dict(torch.load('trained_model.pt'), strict=False)
#         model
#         patched_model = patch_model(model,
#                                   memristor_model=reference_memristor,
#                                   memristor_model_params=reference_memristor_params,
#                                   module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
#                                   mapping_routine=naive_map,
#                                   transistor=True,
#                                   programming_routine=None,
#                                   p_l=None,
#                                   scheme=memtorch.bh.Scheme.DoubleColumn)
#         patched_model = model_degradation(patched_model, cycle_count, v_stop)
#         patched_model = minimal_tune(patched_model)
#         accuracy = test(patched_model.eval(), test_loader)
#         del patched_model
#         del model
#         df = df.append({'times_reprogramed': time_to_reprogram, 'v_stop': v_stop, 'test_set_accuracy': accuracy}, ignore_index=True)
#         df.to_csv('endurance_gradual.csv', index=False)
#         if SAVE_GOOGLE_COLAB:
#           shutil.copy("endurance_gradual.csv", "/content/gdrive/My Drive/endurance_gradual.csv")


## Device endurance (sudden) simulations

In [ ]:
# import enum
# from enum import Enum, auto
# from memtorch.mn.Module import supported_module_parameters
# import math
# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from memtorch.mn.Module import patch_model
# from memtorch.map.Module import naive_tune
# from memtorch.map.Parameter import naive_map
# from memtorch.bh.crossbar.Program import naive_program
# from memtorch.bh.nonideality.NonIdeality import apply_nonidealities
# from memtorch.bh.crossbar.Crossbar import init_crossbar
# import copy
# from mobilenetv2 import MobileNetV2
# from scipy.interpolate import interp1d
# import torchvision


# def minimal_tune(model):
#     for i, (name, m) in enumerate(list(model.named_modules())):
#         if hasattr(m, 'tune'):
#             m.transform_output = lambda input: input
#             if isinstance(m, memtorch.mn.Conv2d):
#                 try:
#                     m.transform_output = naive_tune(m, (4, m.in_channels, 8, 8))
#                 except:
#                     pass
#             if isinstance(m, memtorch.mn.Linear):
#                 try:
#                     m.transform_output = naive_tune(m, (64, m.in_features))
#                 except:
#                     pass
                
#     return model.to(device)
    
# def update_patched_model(patched_model, model):
#     for i, (name, m) in enumerate(list(patched_model.named_modules())):
#         if isinstance(m, memtorch.mn.Conv2d) or isinstance(m, memtorch.mn.Linear):
#             pos_conductance_matrix, neg_conductance_matrix = naive_map(getattr(model, name).weight.data, r_on, r_off,scheme=memtorch.bh.Scheme.DoubleColumn)
#             m.crossbars[0].write_conductance_matrix(pos_conductance_matrix, transistor=True, programming_routine=None)
#             m.crossbars[1].write_conductance_matrix(neg_conductance_matrix, transistor=True, programming_routine=None)
#             m.weight.data = getattr(model, name).weight.data
            
#     return patched_model

# scale_input = interp1d([1.3, 1.9], [0, 1])
# def scale_p_0(p_0, p_1, v_stop, cell_size=10):
#     scaled_input = scale_input(v_stop)
#     x = 1.45
#     y = p_0 * cell_size + p_1
#     k = math.log10(y) / (1 - (2 * scale_input(x) - 1) ** (2))
#     new_y = 10 ** (k * (1 - (2 * scaled_input - 1) ** (2)))
#     p_0 = (new_y - p_1) / cell_size
#     return p_0
       
# def model_sudden(layer, cycle_count, v_stop):
#     cell_size = 10
#     p_1 = 0
#     p_0 = 2e7 / cell_size
#     p_0 = scale_p_0(p_0, p_1, v_stop)
#     threshold = p_0 * cell_size
#     if cycle_count > threshold:
#         for i in range(len(layer.crossbars)):
#             input = layer.crossbars[i].conductance_matrix
#             input[input < (1 / 2e4)] = 1 / 2e4
#             layer.crossbars[i].conductance_matrix = input
            
#     return layer

# def model_degradation(model, cycle_count, v_stop):
#     for i, (name, m) in enumerate(list(model.named_modules())):
#         if type(m) in supported_module_parameters.values():
#             setattr(model, name, model_sudden(m, cycle_count, v_stop))
             
#     return model

# device = torch.device('cuda')
# batch_size = 64
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
# train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, num_workers=1)
# test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=1)
# reference_memristor = memtorch.bh.memristor.VTEAM
# r_on = 2.00e4
# r_off = 10.75e4
# reference_memristor_params = {'time_series_resolution': 1e-10,
#                               'r_off': r_off,
#                               'r_on': r_on}
# times_to_reprogram = 10 ** np.arange(1, 10, dtype=np.float64)
# v_stop_values = np.linspace(1.3, 1.9, 10, endpoint=True)
# df = pd.DataFrame(columns=['times_reprogramed', 'v_stop', 'test_set_accuracy'])
# for time_to_reprogram in times_to_reprogram:
#     cycle_count = len(train_loader.dataset) * time_to_reprogram
#     for v_stop in v_stop_values:
#         print('time_to_reprogram: %f, v_stop: %f' % (time_to_reprogram, v_stop))
#         model = MobileNetV2().to(device)
#         model.load_state_dict(torch.load('trained_model.pt'), strict=False)
#         model.eval()
#         patched_model = patch_model(model,
#                                   memristor_model=reference_memristor,
#                                   memristor_model_params=reference_memristor_params,
#                                   module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
#                                   mapping_routine=naive_map,
#                                   transistor=True,
#                                   programming_routine=None,
#                                   p_l=None,
#                                   scheme=memtorch.bh.Scheme.DoubleColumn)
#         patched_model = model_degradation(patched_model, cycle_count, v_stop)
#         patched_model = minimal_tune(patched_model)
#         accuracy = test(patched_model, test_loader)
#         del patched_model
#         del model
#         df = df.append({'times_reprogramed': time_to_reprogram, 'v_stop': v_stop, 'test_set_accuracy': accuracy}, ignore_index=True)
#         df.to_csv('endurance_sudden.csv', index=False)
#         if SAVE_GOOGLE_COLAB:
#           shutil.copy("endurance_sudden.csv", "/content/gdrive/My Drive/endurance_sudden.csv")

## Device retention (gradual) simulations

In [ ]:
# import enum
# from enum import Enum, auto
# import math
# import pandas as pd
# from memtorch.mn.Module import supported_module_parameters
# import math
# from memtorch.mn.Module import patch_model
# from memtorch.map.Module import naive_tune
# from memtorch.map.Parameter import naive_map
# from memtorch.bh.crossbar.Program import naive_program
# from memtorch.bh.nonideality.NonIdeality import apply_nonidealities
# from memtorch.bh.crossbar.Crossbar import init_crossbar
# import copy
# from mobilenetv2 import MobileNetV2
# from scipy.interpolate import interp1d
# import torchvision


# class OperationMode(Enum):
#     sudden = auto()
#     gradual = auto()

    
# def minimal_tune(model):
#     for i, (name, m) in enumerate(list(model.named_modules())):
#         if hasattr(m, 'tune'):
#             m.transform_output = lambda input: input
#             if isinstance(m, memtorch.mn.Conv2d):
#                 m.transform_output = naive_tune(m, (4, m.in_channels, 8, 8))
#             if isinstance(m, memtorch.mn.Linear):
#                 m.transform_output = naive_tune(m, (64, m.in_features))
                
#     return model.to(device)
    
# def gradual(input, x, p_0, p_1, p_2, p_3, cell_size, tempurature):
#     p_0 = p_0 / input
#     if tempurature > 298:
#         input = torch.pow(10, p_0 * tempurature + torch.log10(input) - p_0 * 298)
        
#     p_0 = torch.log10(input)
#     threshold = p_1 * np.exp(p_2 * cell_size)
#     return torch.pow(10, (p_3 * cell_size * math.log10(x) + torch.log10(10 **  p_0) - p_3 * cell_size * math.log10(threshold)))
    
# def model_gradual(layer, x, tempurature):
#     cell_size = 10
#     convergence_point = 180000
#     p_0_lrs = 0.000801158151673717 * 2400
#     p_0_hrs = 0.00420717061486765 * 55000
#     p_1 = 0.5
#     p_2 = 0.7600902459542083
#     p_3_lrs = 0.006489105105825544  
#     p_3_hrs = -0.007240917429683966
#     threshold_lrs = p_1 * math.exp(p_2 * cell_size)
#     threshold_hrs = p_1 * math.exp(p_2 * cell_size)
#     for i in range(len(layer.crossbars)):
#         input = 1 / layer.crossbars[i].conductance_matrix
#         if input[input < convergence_point].nelement() > 0:
#             if x > threshold_lrs:
#                 input[input < convergence_point] = gradual(input[input < convergence_point], time_, p_0_lrs, p_1, p_2, p_3_lrs, cell_size, tempurature)
#         if input[input > convergence_point].nelement() > 0:
#             if x > threshold_hrs:
#                 input[input > convergence_point] = gradual(input[input > convergence_point], time_, p_0_hrs, p_1, p_2, p_3_hrs, cell_size, tempurature)

#         layer.crossbars[i].conductance_matrix = 1 / input

#     return layer
    
# def model_degradation(model, time_, operation_mode, tempurature):
#     for i, (name, m) in enumerate(list(model.named_modules())):
#         if type(m) in supported_module_parameters.values():
#             if len(name.split('.')) > 1:
#                 name = name.split('.')[1]

#             if operation_mode == OperationMode.gradual:
#                 if hasattr(model, 'module'):
#                     setattr(model.module, name, model_gradual(m, time_, tempurature))
#                 else:
#                     setattr(model, name, model_gradual(m, time_, tempurature))
#             elif operation_mode == OperationMode.sudden:
#                 if hasattr(model, 'module'):
#                     setattr(model.module, name, model_sudden(m, time_, tempurature))
#                 else:
#                     setattr(model, name, model_sudden(m, time_, tempurature))
                    
#     return model

# device = torch.device('cuda')
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])
# train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, num_workers=1)
# test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=1)
# reference_memristor = memtorch.bh.memristor.VTEAM
# r_on = 9e4
# r_off = 330000
# reference_memristor_params = {'time_series_resolution': 1e-10,
#                               'r_off': r_off,
#                               'r_on': r_on}
# times = 10 ** np.arange(1, 10, dtype=np.float64)
# tempuratures = np.linspace(75, 175, 10, endpoint=True)
# df = pd.DataFrame(columns=['time', 'tempurature', 'test_set_accuracy'])
# for time_ in times:
#     print(time_)
#     for tempurature in tempuratures:
#         tempurature += 273
#         model = MobileNetV2().to(device)
#         model.load_state_dict(torch.load('trained_model.pt'), strict=False)
#         model.eval()
#         patched_model = patch_model(model,
#                                   memristor_model=reference_memristor,
#                                   memristor_model_params=reference_memristor_params,
#                                   module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
#                                   mapping_routine=naive_map,
#                                   transistor=True,
#                                   programming_routine=None,
#                                   p_l=None,
#                                   scheme=memtorch.bh.Scheme.DoubleColumn)

#         patched_model = minimal_tune(patched_model)
#         patched_model = model_degradation(patched_model, time_, OperationMode.gradual, tempurature)
#         accuracy = test(patched_model, test_loader)
#         del patched_model
#         df = df.append({'time': time_, 'tempurature': tempurature, 'test_set_accuracy': accuracy}, ignore_index=True)
#         df.to_csv('retention_gradual.csv', index=False)
#         if SAVE_GOOGLE_COLAB:
#           shutil.copy("retention_gradual.csv", "/content/gdrive/My Drive/retention_gradual.csv")